In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/project/':
    os.chdir('/project/')

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
# data_params = {
#     'data_type': 'simulated_data',
#     'model': {
#         'model_select': 'log_gpm_norm',
#         'model_params':{
#             'H': 10
#         },
#         'inference_params':{
#             'I_y': 0.1,
#             'I_z': 0.1,
#             'Q': 3e13,
#             'sigma': 1
#         }
#     },
#     'domain':{
#         'domain_select': 'cone_from_source_z_limited',
#         'domain_params': {
#             'r': 100,
#             'theta': np.pi/8,
#             'source': [0,0,10]
#         },
#         'resolution': 20
#     },
#     'noise_dist': 'gaussian',
#     'noise_level': 1,
#     'output_header': 'Concentration',
# }

data_params = {
    'data_type': 'normalised_data',
    'data_select': 'GBR_data',
    'normaliser_select': 'GBR_normaliser',
    'normaliser_params':{
        'experiments_list': [
            'Exp1',
            'Exp2',
            'Exp3',
            'Exp4',
            'Exp5',
            'Exp6',
            'Exp7',
            'Exp8',
            'Exp9',
            'Exp10',
            'Exp11',
            'Exp12',
        ],
        'meta_data_select': 'GBR_data_summary',
        'input_header': 'Concentration'
    },
    'log':True,
    'output_header': 'Concentration',
    'gridding': [100,100,25]
}



default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y': Parameter('I_y', prior_select = 'log_norm').add_prior_param('mu', 0.1).add_prior_param('sigma', 0.1),
            'I_z': Parameter('I_z', prior_select = 'log_norm').add_prior_param('mu', 0.1).add_prior_param('sigma', 0.1),
            'Q': Parameter('Q', prior_select = 'log_norm').add_prior_param('mu', 3.5e13).add_prior_param('sigma',1e7),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.25).add_prior_param('sigma',0.7)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',10),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 1000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

prior_plots = [
    {
        'I_y': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'labels': ['A','B','C','D','E','F'] 
        }
    },
    {
        'I_z': [0.0001, 1.7],
        'references':{
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    },
    {
        'Q': [1e12, 5e13],
        'references':{
            'Q' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }
    },
    {
        'sigma': [0.001, 4],
    },
    {
        'I_y': [0.0001, 1.7],
        'I_z': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    }
]

results_name = 'test_3'

In [3]:
sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()

visualiser.get_prior_plots(prior_plots)


domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 100)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])

visualiser.visualise_results(domain = domain, name = 'small_scale_3D_plots', title='Log Concentration of Droplets', log_results=False)
visualiser.animate(name = 'small_scale_3D_plots')

sample: 100%|██████████| 1250/1250 [00:00<00:00, 5687.96it/s, 7 steps of size 6.34e-01. acc. prob=0.91]


Creating instance


In [4]:
# generator = Generator(results_name=results_name, 
#                   data_params=data_params,
#                   default_params=default_params)

In [5]:
# analysis_iterations = {
#     'parameters_1':
#     [
#         'sigma_mu'
#     ],
#     'parameters_2':
#     [
#         'sigma_sigma'

#     ],
#     'values_1':
#     [
#         np.array([0.5, 1])     
#     ],

#     'values_2':
#     [
#         np.array([0.5, 1]) 
#     ]
# }

# for i in range(len(analysis_iterations['parameters_1'])):
#     parameter_1 = analysis_iterations['parameters_1'][i]
#     parameter_2 = analysis_iterations['parameters_2'][i]

#     print('Working on varying ' + parameter_1 + '...')
#     values_1 = analysis_iterations['values_1'][i]
#     values_2 = analysis_iterations['values_2'][i]
#     inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1='linear', scale_2='linear', plot=True)

## Optimiser ##

Optimise the default construction using Bayesian Optimisation

In [6]:
optimising_parameters = {
                    'I_y_mu': [1e-2, 2],
                    'I_y_sigma': [1e-2, 2],
                    'I_z_mu': [1e-2, 2],
                    'I_z_sigma': [1e-2, 2],
                    'Q_mu': [1e12, 1e14],
                    'Q_sigma': [1e6, 1e7],
                    'sigma_mu':[0.2, 2],
                    'sigma_sigma':[0.2, 2]
                }

optimiser = Optimiser(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

study = optimiser.run(n_trials=3, optimiser_name='AIC_1', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)
domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,10])
optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

# study = optimiser.run(n_trials=50, optimiser_name='AIC_2', optimising_parameters=optimising_parameters, index_name='aic')
# optimiser.get_plots(study)
# domain = Domain('cone_from_source_z_limited', resolution=80)
# domain.add_domain_param('r', 1000)
# domain.add_domain_param('theta', np.pi/8)
# domain.add_domain_param('source', [0,0,10])
# optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

# study = optimiser.run(n_trials=50, optimiser_name='AIC_3', optimising_parameters=optimising_parameters, index_name='aic')
# optimiser.get_plots(study)
# domain = Domain('cone_from_source_z_limited', resolution=80)
# domain.add_domain_param('r', 1000)
# domain.add_domain_param('theta', np.pi/8)
# domain.add_domain_param('source', [0,0,10])
# optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)


[I 2023-10-17 00:22:03,605] A new study created in RDB with name: AIC_1
sample: 100%|██████████| 1250/1250 [00:00<00:00, 5233.38it/s, 7 steps of size 6.18e-01. acc. prob=0.92]
[I 2023-10-17 00:22:06,644] Trial 0 finished with value: 64.48651885986328 and parameters: {'I_y_mu': 0.8121068805817191, 'I_y_sigma': 0.25673754719701825, 'I_z_mu': 1.1799086967372074, 'I_z_sigma': 1.9053316396143367, 'Q_mu': 26448454462357.785, 'Q_sigma': 4571598.704720803, 'sigma_mu': 1.0953082253378699, 'sigma_sigma': 0.4631540245901231}. Best is trial 0 with value: 64.48651885986328.
sample: 100%|██████████| 1250/1250 [00:00<00:00, 5389.36it/s, 7 steps of size 3.88e-01. acc. prob=0.91]
[I 2023-10-17 00:22:09,662] Trial 1 finished with value: 66.60711669921875 and parameters: {'I_y_mu': 1.2183471153609826, 'I_y_sigma': 1.3740521186494408, 'I_z_mu': 1.9165649896691352, 'I_z_sigma': 0.3060815199565892, 'Q_mu': 82111222416249.23, 'Q_sigma': 7650530.992477362, 'sigma_mu': 0.7093210987483143, 'sigma_sigma': 1.1067